# Task description

Shortend RU version. No blablaing. TO BE WRITTEN

Context

1. ...

2. Task conditions - statements

3. Math

4. Remarks

## 1. Предисловие

## 2. Формулировка задачи - постулаты

### "Классическая"  задача  VRPMT

Для задачи "Vehicle Routing Problem with Multiple Trips" буду использовать формулировку из статьи Diego Cattaruzza и др. 

Let G = (N,A) be a directed graph, where N = {0,1,...,N} is the set of nodes and A = {(i,j)|i,j ∈N} is the set of arcs. Arcs (i,j) ∈A are characterized by their travel time Tij. Node 0 represents the depot where a ﬂeet V of identical vehicles with limited capacity Q is available at time 0 and has to be returned at time TH. Nodes 1,...,N represent the customers to be served, each one requiring a certain non-negative quantity Qi of a product. The MTVRP calls for the determination of a set of trips and an assignment of each trip to a vehicle, such that the traveled time is minimized and the following conditions are satisﬁed:

(i) each trip starts and ends at the depot,

(ii) each customer is visited exactly once,

(iii) the sum of the demands of the customers in any trip does not exceed Q,

(iv) the sum of the durations of the trips assigned to the same vehicle (journey) does not exceed TH (a trip duration being the sum of the travel times on arcs used in the trip).

Для моей задачи должно быть добавлено еще условие посещения узлов в определенный интервал времени.

### Отличие моей задачи от классической

Итак ориентируемся на условия из Предисловия и смотрим на отличия.

_1) Такого нет в классической задаче:_

- В одном городе может быть несколько заказов.

- Заказ характризуется требуемым объемом, ценой по которой заказчик готов купить нефть и временным окном доставки. 

Немного подумав становиться понятным, что здесь нет большой проблемы. Каждый заказ можно рассматривать, как отдельный город - добавляем вершины в граф. Для заказов в одном городе добавляем в граф ребра, соединяющие добавленные вершины между собой, с нулевыми длинной и временем движения вдоль них. Для каждой новой вершины добавлем старые ребра с их характеристиками.

Далее я считаю, что в каждом городе сидит только один заказ. Отдельно рассмотрю случай в одном городе несколько заказов.

_Замечание:_

Замена заказов на города может привести к "взрыву" в количестве переменных. На практике еще не пробовал. Альтернатива - вводить переменные отслеживающте сколько заказа исполнено. 

_2) В классической задаче каждый заказчик посещен только один раз, только одним ТС за весь период._

В моем случае заказчик посещен один раз одиним ТС только за один рейс, т.е. ТС посетило заказчикаБ исполнило заказ частично, вернулось на базу, заправилось и может ехать к этому заказчику опять. 

_3) В классической задаче у всех машин одинаковая вместимость._

Это мелочь. В классической задаче не проблема рассматривать ТС разной вместимости. Упомянул для проформы.

_4) В классичекой задаче сумма спроса у заказчиков не превышает вместимость ТС._

В моей постановке такое может быть. Это сильное отличие. Именно из-за этого у меня есть 1). СОВМЕСТИТЬ!!! 1) и 4) 

### Окончательная формулировка моей задачи

- Дан набор транспортных стредств (ТС) разной вместимости. 

- Требуется доставить нефть из базы, заказчикам в разных городах. 

- **В одном городе находится 1 заказ (случай нескольких заказов рассмотрю отдельно).** 

- Заказ характризуется требуемым объемом, ценой по которой заказчик готов купить нефть и временным окном доставки.

- В начальный момент времени все ТС находятся на базе. 

- В последний момент времени все ТС также должны быть на базе. 

- Загрузка данного ТС возможна только на базе. 

- Временем на загрузку или разгрузку ТС пренебрегаем.

- Заданы расстояния между городами, стоимость за 1 км пути (для каждого ТС своя), время движения между городами (не зависит от ТС).

- Величина отдельного заказа может превашать вместимость любого ТС.

- **Очень важно!!!**

    1. Допустимо, что заказ может быть выполен не полностью или вообще не выполнен.
    2. Данное ТС может посетить данный город за один рейс только один раз, но не запрещается посещять этот город в другие рейсы. 
    3. Каждый город, где есть хоть один заказ должен быть посещен хотя бы один раз, хотя бы одним ТС.

    4. _Внимание!!! Допустим есть город, в который можно приехать и вернуться только через один другой город. 
    КАЖЕТСЯ!!!Для такого города эти 3 условия делают задачу не решабельной (infeasible). ПРОВЕРИТЬ!!!

- Требуется максимизировать операционную прибыль = выручка от продажи нефти заказчикам минус затраты на доставку. Также желательно, чтобы ТС возвращались на базу как можно быстрее.

## 3. Математическое описание

В этой части я использую обозначения, примерно как в  Diego Cattaruzza и др. В моих питоновских программах обозначения немного другие, но тоже похожи на Diego Cattaruzza и др.

### Начальные данные

- $G = (N_0, A)$ - ориентированный граф, где  $N_0 = {0, 1, ..., N}$ набор вершин, а $A = \{(i, j) |\ i, j  \in N\}$ набор ребер.
- Известна длинна каждого ребра, $a_{ij}$.
- На каждом ребре задано время движения по нему $T_{ij}$. 
- Считаем, что база находится в вершине 0.
- Покупатели (спрос) находятся в вершинах $N = N_0 \setminus \{0\}$. 
- $V$ - набор различных ТС, причем у каждого своя вместимость $Q_v$ и своя стоимость за единицу пути $C_v$.
- Величина спроса для данной вершины равна $D_i$ единиц, цена за единицу $P_i$, спрос должен быть удовлетворен во временнном интервале $[T^{(1)}_i, T^{(2)}_i]$,  где $i\in N$.
- Начальный момент времени, когда все ТС на базе, равен 0.
- Конечный момент времени, к которому все машины должны вернуться на базу, $T_H$.

### Переменные

Пусть $R$ - количество рейсов которое может совершить каждое из имеющихся ТС. Это число заранее неизвестно, но может быть оценено. В классической VRPMT задаче его можно положить равным $N$. Здесь я его оцениваю, как $R = [\sum D_i/\sum Q_v] + 1$, т.е. количество рейсов это $\{1..R\}$.

$$
x^{vr}_{ij} = 
\begin{cases}
1, \text{если для рейса }r \in R, \text{ТС } v \in V, \text{проехало по ребру }(i, j) \in A,\\
0, иначе
\end{cases}
$$

$$
y^{vr}_{i} = 
\begin{cases}
1, \text{если для рейса }r \in R, \text{ ТС } v \in V, \text{посетило вершину }i \in N_0,\\
0, иначе
\end{cases}
$$

$t^{vr}_{i} \ge 0 -$  время посещения вершины $i \in N_0$, во время рейса $r \in R, v \in V$.

$t^{vr}_{i} = 0 -$ если ТС не посетило вершину.

$t^{vr}_{H} -$  время возврата ТС на базу по окончании рейса $r \in R$.

$d^{vr}_{i}\ge 0 - $ количество продукта доставленное ТС  $v \in V$, в вершину  $i \in N$.

$L_i \ge 0 -$ недопоставленное количество продукта в вершину $i \in N$. В ограничениях используется, чтобы разрешить неполностью удовлетворять спрос. В целевой фунции используется совместно с множителем $f_p = 100$, чтобы делать недопоставку невыгодной.

#### Целевая функция - максимизировать прибыль

$$
\begin{split}
    \max & \ \sum_{i \in N} \sum_v \sum_r P_i d^{vr}_i \\
         & - \sum_v \sum_r \sum_{i, j} a_{ij} C_v x^{vr}_{ij} \\
         & - \sum_v \sum_r t^{vr}_{E} \\
         & - \sum_{i \in N} f_p P_i L_i 
\end{split}
$$
Здесь первое слагаемое выручка за доставленное сырье. Второе - затраты на доставку. Третье слагаемое заставляет возвращаться на базу, как можно раньше. Четвертое слагаемое - делает недопоставку невыгодной.

#### Ограничения на переменные связанне со временем

$$
\begin{eqnarray*}
    t^{v1}_0 = 0 & \ \ \ & \forall v \in V,  & \ \ \ (1)\\
    t^{vr}_{H} \le t^{vr+1}_0 & \ \ \ & \forall r \in \{1..R-1\}, \forall v \in V & \ \ \  (2)\\
    t^{vr}_0   \le t^{vr+1}_0 & \ \ \ & \forall r \in \{1..R-1\}, \forall v \in V & \ \ \  (3)\\
    t^{vr}_{H} \le T_H & \ \ \ & \forall r \in \{1..R\}, \forall v \in V & \ \ \  (4)\\
    t^{vr}_i + T_{ij} \le t^{vr}_j + T_H (1 - x^{vr}_{ij}) & \ \ \ & 
                \forall r \in \{1..R\}, \forall v \in V, \forall i \in N_0, \forall j \in N & \ \ \  (5)\\
    t^{vr}_i + T_{i0} \le t^{vr}_H + T_H (1 - x^{vr}_{i0}) & \ \ \ & 
                \forall r \in \{1..R\}, \forall v \in V, \forall i \in N_0, \forall j \in N & \ \ \  (6)\\
    T^{(1)}_i y^{vr}_i \le t^{vr}_i \le T^{(2)}_i y^{vr}_i & \ \ \ & \forall r \in \{1..R\}, \forall v \in V, \forall i \in N & \ \ \ (7)\\
\end{eqnarray*}
$$

(1) Начинаем в нулевой момент времени на базе.

(2) Время прибытия на базу меньше, чем время начала следующего рейса.

(3) Время начала рейса меньше, чем время начала ЛЮБОГО следующего рейса. Это казалось бы очевидно, т.к. есть условия (2) и (6). 
   На самом деле это условие запрещает одновременность рейсов через 1. Допустим $R = 4$. Действительно рейс $r=1$ не начнется раньше, чем $r = 2$, но вот $r = 3$, без этого условия, может начаться одновременно с $r=1$. Это условие обнаружено эмперически.

(4) Ограничение на время возврата на базу, для любого рейса.

(5) и (6) Ограничения на время отправки из одной вершины и прибытия в другую.

(7) Ограничения на время выполнения заказа.

#### Ограничения на переменные связанные с перемещениями и выполнениями заказов

$$
\begin{eqnarray*}
    \sum_v \sum_r y^{vr}_i \ge 1       & \ \ \ & \forall i \in N & \ \ \  (8)\\
    \sum_v \sum_r d^{vr}_i = D_i - L_i & \ \ \ & \forall i \in N & \ \ \  (9)\\
    \sum_{j \in N_0} x^{vr}_{ij} = \sum_{j \in N_0} x^{vr}_{ji} = y^{vr}_i & \ \ \ & 
            \forall r \in \{1..R\}, \forall v \in V, \forall i \in N & \ \ \ (10)\\
    \sum_{i \in N} d^{vr}_i \le Q_v & \ \ \ & \forall r \in \{1..R\}, \forall v \in V & \ \ \ (11)\\
    d^{vr}_i \le Q_v y^{vr}_ i& \ \ \ & \forall r \in \{1..R\}, \forall v \in V, \forall i \in N & \ \ \ (12)\\
\end{eqnarray*}
$$

(8) Каждая вершина, где есть хоть один заказ должен быть посещен хотя бы один раз, хотя бы одним ТС.
Не уверен, что это условие нужно. ПРОВЕРИТЬ!!!

(9) Удовлетворить заказ в данной вершине, если возможно.

(10) Поток -  если ТС приехало в вершины, то должно и уехать.

(11) Доставленное сырье за одтн рейс не превышает вместимость ТС.

(12) Связь между доставленным сырьем и посещением вершины.

## 4. Замечания

Не отвечаю за правильность. Может упустил что-то.

Интересна матемаматическая лишь формулировка, т.е. не знаю решиться ли эта задача стандарным софтом, когда городов больше 100 и не предлагаю алгоритма. Это выходит за рамки данного проекта